# Bird Classification with PyTorch
## Table of Contents
* 1. Set up
* 2. Import datasets
* 3. Model Development
* 4. Model Evaluation
* 5. Make Predictions
* 6. Save the Model

## 1. Set up

In [ ]:
import torchvision
import torch
import numpy as np
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time
import pandas as pd

## 2. Import datasets

In [ ]:
train_path = "/kaggle/input/100-bird-species/train"
val_path = "/kaggle/input/100-bird-species/valid"
test_path = "/kaggle/input/100-bird-species/test" 
batch_size = 64
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_folder = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
valid_folder = torchvision.datasets.ImageFolder(root=val_path, transform=transform)
test_folder = torchvision.datasets.ImageFolder(root=test_path, transform=transform)
train_ds = DataLoader(train_folder, batch_size=batch_size)
valid_ds = DataLoader(valid_folder, batch_size=batch_size)
test_ds = DataLoader(test_folder, batch_size=batch_size)

## 3. Model Development

### 3.1 The CNN Model

In [ ]:
class CNNModel(nn.Module):
    
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
n_classes = len(train_folder.classes)
model = CNNModel(n_classes)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)

### 3.2 Training

In [ ]:
epochs = 10
train_steps = len(train_ds)
valid_steps = len(valid_ds)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {"loss": [], "val_loss": [], "val_accuracy": []}
for epoch in range(epochs):
    train_losses = []
    valid_losses = []
    model.train()
    begin = time.time()
    for batch in train_ds:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.data.item())
        if len(train_losses) > 0 and len(train_losses) % 20 == 0:
            current = time.time()
            elapsed = current - begin
            print("Epoch %d: [Training] %.2fs/%.2fs"%(epoch + 1, elapsed, elapsed / float(len(train_losses)) * train_steps))
    model.eval()
    num_correct = 0
    num_samples = 0
    begin = time.time()
    for batch in valid_ds:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        loss = loss_fn(output, targets)
        valid_losses.append(loss.data.item())
        correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
        num_correct += torch.sum(correct).item()
        num_samples += correct.shape[0]
        if len(valid_losses) > 0 and len(valid_losses) % 10 == 0:
            current = time.time()
            elapsed = current - begin
            print("Epoch %d: [Validation] %.2fs/%.2fs"%(epoch + 1, elapsed, elapsed / float(len(valid_losses)) * valid_steps))
    train_loss = torch.mean(torch.Tensor(train_losses)).item()
    valid_loss = torch.mean(torch.Tensor(valid_losses)).item()
    accuracy = num_correct / num_samples if num_samples > 0 else 0
    metrics["loss"].append(train_loss)
    metrics["val_loss"].append(valid_loss)
    metrics["val_accuracy"].append(accuracy)
    print("Training Loss: %.2f Validation Loss: %.2f accuracy: %.2f" %(train_loss, valid_loss, accuracy))

## 4. Model Evaluation

### Loss & Accuracy over time

In [ ]:
pd.DataFrame(metrics).plot()

## 5. Make Predictions

In [ ]:
np.random.choice(len(test_folder), 1)

In [ ]:
labels = train_folder.classes
for i in np.random.choice(len(test_folder), 30):
    item, actual = test_folder[i]
    prediction = model(item.view(-1, 3, 64, 64).to(device)).argmax()
    print("Prediction: %s Actual result: %s"%(labels[prediction], labels[actual]))

## 6. Save the Model

In [ ]:
path = "bird_classfier"
torch.save(model, path)

## 7. Load the Model

In [ ]:
model = torch.load(path)